In [ ]:
import pandas as pd
from src.train import train_loop
from src.model import CNNLSTMModel
from src.dataset import InverterTimeSeriesDataset
import torch

inverter_data = pd.read_parquet('data/inverter_data/')
inverter_data['event_local_time'] = pd.to_datetime(inverter_data['event_local_time'])
failure_sessions = pd.read_csv('data/failure_sessions.csv', index_col=0, parse_dates=['start_time', 'end_time'])

In [2]:
## TO BE REPLACED
inverter_data['label'] = 0
#sample_data = inverter_data[inverter_data['device_name'] == 'INV 51'].copy()
sample_data = inverter_data.copy()
sample_data.fillna(0, inplace=True)

In [3]:
feature_cols = ['metric.AC_CURRENT_A.MEASURED', 'metric.AC_POWER.MEASURED']
dataset = InverterTimeSeriesDataset(sample_data, feature_cols)

Processing devices: 100%|██████████| 16/16 [00:04<00:00,  3.36it/s]


In [4]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

model = CNNLSTMModel(num_features=len(feature_cols))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

In [5]:
train_loop(model, train_loader, num_epochs=10, optimizer=optimizer, criterion=criterion)

[Epoch 1/10] Step 0/191359 - Loss: 0.2779
[Epoch 1/10] Step 100/191359 - Loss: 0.0014
[Epoch 1/10] Step 200/191359 - Loss: 0.0001
[Epoch 1/10] Step 300/191359 - Loss: 0.0001
[Epoch 1/10] Step 400/191359 - Loss: 0.0000
[Epoch 1/10] Step 500/191359 - Loss: 0.0000
[Epoch 1/10] Step 600/191359 - Loss: 0.0000
[Epoch 1/10] Step 700/191359 - Loss: 0.0000
[Epoch 1/10] Step 800/191359 - Loss: 0.0000
[Epoch 1/10] Step 900/191359 - Loss: 0.0000
[Epoch 1/10] Step 1000/191359 - Loss: 0.0000
[Epoch 1/10] Step 1100/191359 - Loss: 0.0000
[Epoch 1/10] Step 1200/191359 - Loss: 0.0000
[Epoch 1/10] Step 1300/191359 - Loss: 0.0000
[Epoch 1/10] Step 1400/191359 - Loss: 0.0000
[Epoch 1/10] Step 1500/191359 - Loss: 0.0000
[Epoch 1/10] Step 1600/191359 - Loss: 0.0000
[Epoch 1/10] Step 1700/191359 - Loss: 0.0000
[Epoch 1/10] Step 1800/191359 - Loss: 0.0000
[Epoch 1/10] Step 1900/191359 - Loss: 0.0000
[Epoch 1/10] Step 2000/191359 - Loss: 0.0000
[Epoch 1/10] Step 2100/191359 - Loss: 0.0000
[Epoch 1/10] Step 2200

KeyboardInterrupt: 